# Agent training stability
Check if reward is stable for the same training but with different random seeds.

In [ ]:
import os
import neptune.new as neptune
import pandas as pd
import seaborn as sns

sns.set(style="darkgrid")

## Fetch project

In [ ]:
project = neptune.get_project(name="common/project-rl")

## Fetch runs with "seed" tag

In [ ]:
runs_table_df = project.fetch_runs_table(tag="seed").to_pandas()

In [ ]:
stability_df = pd.DataFrame(columns=["sys/id", "step", "value"])

## Fetch "reward" data from runs

In [ ]:
for index, row in runs_table_df.iterrows():

    # Resume run in read-only mode
    run = neptune.init(
        api_token=os.getenv("NEPTUNE_API_TOKEN"),
        project="common/project-rl",
        run=row["sys/id"],
        mode="read-only",
    )

    # Fetch values for the reward
    reward_df = run["training/episode/reward"].fetch_values()

    reward_df = reward_df[["step", "value"]]
    reward_df.insert(loc=0, column="sys/id", value=row["sys/id"])
    stability_df = stability_df.append(reward_df, sort=True)

    # Stop the run
    run.stop()

stability_df = stability_df.rename(columns={"value": "reward"})

## Visualize reward with error bars

In [ ]:
g = sns.relplot(
    x="step",
    y="reward",
    kind="line",
    ci="sd",
    height=9,
    aspect=1,
    data=stability_df[["step", "reward"]])